### 5.1.1 自定义快

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params(稍后将介绍)
        super.__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层
    
    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义
        return self.out(F.relu(self.hidden(X)))

C:\Users\25466\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\25466\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\25466\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [11]:
m = nn.Sequential(nn.Linear(4, 8))

nn.init.uniform_(m[0].weight, -10, 10)
print(m[0].weight)
m[0].weight.data.abs() >= 5


Parameter containing:
tensor([[ 1.3321, -3.2149,  9.2920,  5.1054],
        [ 6.8221,  1.8245,  6.1485,  9.9427],
        [-5.2625, -9.2362,  2.0045,  0.8954],
        [ 5.9172,  9.0873, -8.5574, -5.7768],
        [-0.4705,  5.1595,  9.3268,  3.5238],
        [-1.6913,  6.1382,  8.3394,  7.0763],
        [ 5.8912,  4.6128,  3.7132,  2.2217],
        [ 9.7546,  4.2513,  5.7260,  7.4635]], requires_grad=True)


tensor([[False, False,  True,  True],
        [ True, False,  True,  True],
        [ True,  True, False, False],
        [ True,  True,  True,  True],
        [False,  True,  True, False],
        [False,  True,  True,  True],
        [ True, False, False, False],
        [ True, False,  True,  True]])

### 5.5.1 加载和保存张量

In [13]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

### 5.5.2 加载和保存模型参数

In [22]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [25]:
torch.save(net.state_dict(), 'mlp.params')

clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

### 5.6.1 计算设备

In [26]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [27]:
torch.cuda.device_count()

1

### 5.6.2 张量与GPU

In [28]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [30]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [31]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

### 5.6.3 神经网络与GPU

In [34]:
net = nn.Sequential(nn.Linear(3,1))
net = net.to(device=try_gpu())

net(X)

tensor([[0.2996],
        [0.2996]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [35]:
net[0].weight.data.device

device(type='cuda', index=0)

In [51]:
from d2l import torch as d2l
timer = d2l.Timer()
# x = torch.randn(size=(1000,2000), device=try_gpu())
# y = torch.randn(size=(2000,1000), device=try_gpu())
x = torch.randn(size=(1000,2000))
y = torch.randn(size=(2000,1000))
z = torch.mm(x, y)
print(f'{timer.stop():.2f} sec')
z

0.08 sec


tensor([[ 23.5112,  33.4057,  84.1753,  ...,  67.5765,  34.7057, -51.5502],
        [-21.3340, -25.4208,  55.7126,  ..., -39.8087, -93.4398,  -6.2595],
        [ -4.5326,  34.2641,  -1.7103,  ..., -17.5033,  80.1725, -28.3361],
        ...,
        [-17.3557,   7.6349, -82.8638,  ..., -72.4234,  73.4989,  34.6824],
        [  0.5403,  11.6290,   1.2308,  ..., -13.4468,  64.1835, -28.2016],
        [-34.8789,  19.0869,   2.2321,  ...,  -6.5718,  56.2175,   4.1928]])